# Data Wrangling (Netflix data)


<h2>Table of content</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="#data_standardization">Data Summary and Introduction</a></li>
    <li><a href="#identify_handle_missing_values">Identify and handle missing values</a>
        <ul>
            <li><a href="#identify_missing_values">Identify missing values</a></li>
            <li><a href="#deal_missing_values">Deal with missing values</a></li>
            <li><a href="#correct_data_format">Correct data format</a></li>
        </ul>
</ul>
</div>
 
<hr>

<h2>Data Summary and Introduction</h2>
An introduction to the use of big data as a strategic resource. A focus is placed on integrating the knowledge of analytics tools with an understanding of how companies leverage data analytics to gain strategic advantage. A case approach is used to emphasize hands-on learning and a real-world view of big data analytics.

Netflix is a popular streaming service that offers a vast catalog of movies, TV shows, and original contents. This dataset is a cleaned version of the original version which can be found here. The data consist of contents added to Netflix from 2008 to 2021.This dataset will be cleaned with Jupiter notebook/ python and visualized. The purpose of this dataset is to test my data cleaning and visualization skills. The cleaned data will be attached.

#### Importing Libraries

In [29]:
import pandas as pd 
import numpy as np 

#### Import the data and explore data 


In [30]:
df = pd.read_csv('netflix_titles.csv')
df = df.copy()
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


<h3 id="Missing data"> Missing data</h3><b</b>

Steps for working with missing data:
<ol>
    <li>dentify missing data</li>
    <li>deal with missing data</li>
    <li>correct data format</li>
</ol>

In [31]:
df.shape

(8807, 12)

<h4>1. Evaluating Missing Data</h4>


In [32]:
df.isnull().any().any()

True

"True" stands for missing value, while "False" stands for not missing value.



<h4>2. Count missing values in each column</h4>


In [33]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [34]:
df["duration"][df["duration"].isnull()==True]

5541    NaN
5794    NaN
5813    NaN
Name: duration, dtype: object

Based on the summary above, each column has 8807 rows of data, six columns containing missing data:
<ol>
    <li>"director": 2634 missing data</li>
    <li>"cast": 825 missing data</li>
    <li>"country": 831 missing data</li>
    <li>"date_added" : 10 missing data</li>
    <li>"rating": 4 missing data</li>
    <li>"duration": 3 missing data</li>
   
</ol>

<h3 id="deal_missing_values">Deal with missing data</h3>

There are different ways to deal with missing data, as we have discussed. Consider:

<ol>
    <li>Drop data<br>
        a. drop the whole row<br>
        b. drop the whole column
    </li>
    <li>Replace data<br>
        a. replace it by correlation<br>
        b. replace it by frequency<br>
        c. replace it based on other functions
    </li>
</ol>




1. 
(a) The "date_added" rows nulls is just 10 out of over 8000 rows, deleting them cannot affect our analysis or visualization. 

In [35]:
# simply drop whole row with NaN in "date_added" column
before_rows = df.shape[0]
df.dropna(subset=["date_added"], axis=0, inplace=True)
after_rows = df.shape[0]
print("Number of dropped rows {}".format(before_rows - after_rows))
# reset index, because we droped 10 rows
df.reset_index(drop=True, inplace=True)

Number of dropped rows 10


In [36]:
df.shape

(8797, 12)

We can do the same for the "rating" rows (4 missing) as well as the "duration" (3 missing) rows.

In [37]:
# simply drop whole row with NaN in "results" column
before_rows = df.shape[0]
df.dropna(subset=["rating"], axis=0, inplace=True)
after_rows = df.shape[0]
print("Number of dropped rows {}".format(before_rows - after_rows))
# reset index, because we droped 10 rows
df.reset_index(drop=True, inplace=True)

Number of dropped rows 4


In [38]:
df.shape

(8793, 12)

In [39]:
# simply drop whole row with NaN in "results" column
before_rows = df.shape[0]
df.dropna(subset=["duration"], axis=0, inplace=True)
after_rows = df.shape[0]
print("Number of dropped rows {}".format(before_rows - after_rows))
# reset index, because we droped 10 rows
df.reset_index(drop=True, inplace=True)

Number of dropped rows 3


In [40]:
df.shape

(8790, 12)

2. The director column nulls is about 30% of the whole column, therefore I will not delete them. I will rather find another column to populate it. To populate the director column, we want to find out if there is relationship between movie_cast column and director column. Just like the director column, I will not delete the nulls in country. Since the country column is related to director and movie, we are going to populate the country column with the director column.

In [42]:
# Populate 'director' column with 'cast' for null rows
df['director'].fillna(df['cast'], inplace=True)

# Now, 'director' column should be updated
print(df)


     show_id     type                  title  \
0         s1    Movie   Dick Johnson Is Dead   
1         s2  TV Show          Blood & Water   
2         s3  TV Show              Ganglands   
3         s4  TV Show  Jailbirds New Orleans   
4         s5  TV Show           Kota Factory   
...      ...      ...                    ...   
8785   s8803    Movie                 Zodiac   
8786   s8804  TV Show            Zombie Dumb   
8787   s8805    Movie             Zombieland   
8788   s8806    Movie                   Zoom   
8789   s8807    Movie                 Zubaan   

                                               director  \
0                                       Kirsten Johnson   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
2                                       Julien Leclercq   
3                                                   NaN   
4     Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
...                                                 ...   
8785      

In [43]:
df.isnull().sum()

show_id           0
type              0
title             0
director        352
cast            825
country         829
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
description       0
dtype: int64

Populate the rest of the NULL in director and cast as "Not Given"

In [44]:
# Populate 'director' column with 'Not Given' for remaining null rows
df['cast'].fillna('Not Given', inplace=True)
df['director'].fillna('Not Given', inplace=True)
# Now, 'director' column should be updated with 'Not Given' for remaining null rows
print(df)


     show_id     type                  title  \
0         s1    Movie   Dick Johnson Is Dead   
1         s2  TV Show          Blood & Water   
2         s3  TV Show              Ganglands   
3         s4  TV Show  Jailbirds New Orleans   
4         s5  TV Show           Kota Factory   
...      ...      ...                    ...   
8785   s8803    Movie                 Zodiac   
8786   s8804  TV Show            Zombie Dumb   
8787   s8805    Movie             Zombieland   
8788   s8806    Movie                   Zoom   
8789   s8807    Movie                 Zubaan   

                                               director  \
0                                       Kirsten Johnson   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
2                                       Julien Leclercq   
3                                             Not Given   
4     Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
...                                                 ...   
8785      

In [45]:
df.isnull().sum()

show_id           0
type              0
title             0
director          0
cast              0
country         829
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
description       0
dtype: int64

Just like the director column, I will not delete the nulls in country. Since the country column is related to director and movie, we are going to populate the country column with the director column

In [46]:
# Populate null values in the country with the directors 
df['country'].fillna(df['director'], inplace=True)
print(df)


     show_id     type                  title  \
0         s1    Movie   Dick Johnson Is Dead   
1         s2  TV Show          Blood & Water   
2         s3  TV Show              Ganglands   
3         s4  TV Show  Jailbirds New Orleans   
4         s5  TV Show           Kota Factory   
...      ...      ...                    ...   
8785   s8803    Movie                 Zodiac   
8786   s8804  TV Show            Zombie Dumb   
8787   s8805    Movie             Zombieland   
8788   s8806    Movie                   Zoom   
8789   s8807    Movie                 Zubaan   

                                               director  \
0                                       Kirsten Johnson   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
2                                       Julien Leclercq   
3                                             Not Given   
4     Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
...                                                 ...   
8785      

In [47]:
# Populate null values in the country with the directors 
df['country'].fillna(df['director'], inplace=True)
print(df)


     show_id     type                  title  \
0         s1    Movie   Dick Johnson Is Dead   
1         s2  TV Show          Blood & Water   
2         s3  TV Show              Ganglands   
3         s4  TV Show  Jailbirds New Orleans   
4         s5  TV Show           Kota Factory   
...      ...      ...                    ...   
8785   s8803    Movie                 Zodiac   
8786   s8804  TV Show            Zombie Dumb   
8787   s8805    Movie             Zombieland   
8788   s8806    Movie                   Zoom   
8789   s8807    Movie                 Zubaan   

                                               director  \
0                                       Kirsten Johnson   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
2                                       Julien Leclercq   
3                                             Not Given   
4     Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
...                                                 ...   
8785      

In [48]:
df.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [53]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Given,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Julien Leclercq,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,Not Given,Not Given,Not Given,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


We can drop the description and movie_cast column because they are not needed for our analysis or visualization task.

In [54]:
df.drop(['description','cast'],axis=1).head(1)

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries


Some of the rows in country column has multiple countries, for my visualization, we only need one country per row to make my map visualization clean and easy. Therefore, I am going to split the country column and retain the first country by the left which I believe is the original country of the movie

In [56]:
# Split the 'country' column and retain only the first country
df['country'] = df['country'].str.split(',').str[0]
# Now, delete the original 'country' column with multiple entries
df.drop('country', axis=1, inplace=True)
# Now, 'country' column should contain only the first country for each row
print(df)


     show_id     type                  title  \
0         s1    Movie   Dick Johnson Is Dead   
1         s2  TV Show          Blood & Water   
2         s3  TV Show              Ganglands   
3         s4  TV Show  Jailbirds New Orleans   
4         s5  TV Show           Kota Factory   
...      ...      ...                    ...   
8785   s8803    Movie                 Zodiac   
8786   s8804  TV Show            Zombie Dumb   
8787   s8805    Movie             Zombieland   
8788   s8806    Movie                   Zoom   
8789   s8807    Movie                 Zubaan   

                                               director  \
0                                       Kirsten Johnson   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
2                                       Julien Leclercq   
3                                             Not Given   
4     Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
...                                                 ...   
8785      

<h3 id="check_data_format">Check data format</h3>

<p>The last step in data cleaning is checking and making sure that all data is in the correct format (int, float, text or other).</p>

In Pandas, we use:
<p><b>.dtype()</b> to check the data type</p>
<p><b>.astype()</b> to change the data type</p>

Lets list the data types for each column...

In [50]:
df.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [57]:
# Convert to CSV file
df.to_csv('wrangled_netflix_titles_data.csv')